In [1]:
# ====================================================
#  TP Bioinformatique - Alignement (Partie I)
#  Objectif : Télécharger deux séquences par identifiant NCBI
#              et les retourner sous forme de chaînes de caractères
# ====================================================

from Bio import Entrez, SeqIO

# 1️⃣ Configuration de l’accès à NCBI
Entrez.email = "alaegmr98@gmail.com"  # Remplace par ton email

# 2️⃣ Liste des identifiants à télécharger
accessions = ["NM_001101.5", "AC035147.4"]

# 3️⃣ Fonction pour télécharger une séquence GenBank et la retourner en chaîne
def telecharger_sequence(accession_id):
    print(f"🔍 Téléchargement de {accession_id} depuis NCBI...")

    # Requête à NCBI
    handle = Entrez.efetch(
        db="nucleotide", 
        id=accession_id, 
        rettype="gb", 
        retmode="text"
    )

    # Lecture du fichier GenBank
    record = SeqIO.read(handle, "genbank")
    handle.close()

    # Récupération de la séquence brute
    sequence = str(record.seq)

    print(f"✅ Séquence {accession_id} récupérée ({len(sequence)} bases).")
    return sequence

# 4️⃣ Téléchargement des deux séquences
seq1 = telecharger_sequence(accessions[0])
seq2 = telecharger_sequence(accessions[1])

# 5️⃣ Affichage des résultats
print("\n=== SÉQUENCE 1 ===")
print(f"Identifiant : {accessions[0]}")
print(f"Longueur    : {len(seq1)} bases")
print(seq1[:200] + " ...")  # On affiche seulement les 200 premières bases

print("\n=== SÉQUENCE 2 ===")
print(f"Identifiant : {accessions[1]}")
print(f"Longueur    : {len(seq2)} bases")
print(seq2[:200] + " ...")


🔍 Téléchargement de NM_001101.5 depuis NCBI...
✅ Séquence NM_001101.5 récupérée (1812 bases).
🔍 Téléchargement de AC035147.4 depuis NCBI...
✅ Séquence AC035147.4 récupérée (104940 bases).

=== SÉQUENCE 1 ===
Identifiant : NM_001101.5
Longueur    : 1812 bases
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCACACCCGCCGCCAGCTCACCATGGATGATGATATCGCCGCGCTCGTCGTCGACAACGGCTCCGGCATGTGCAAGGCCGGCTTCGCGGGCGACGATGCCCCCCGGGCCGTCTTCCCCTCCATCGTGGGGCGCCCCAG ...

=== SÉQUENCE 2 ===
Identifiant : AC035147.4
Longueur    : 104940 bases
AAGCTTTCCCTATTTAAAAATGTATTGTTAAAGATTTGCTACGTGCCAGACTCTTGCTAAACTCTCGTTATTTCATTGGATCACTAAACAACCCTATGATGTAGGTACTATTATTATTACTGCCATTCACAAATAAAGCTTACAAAGTTACAGCTCAAGGTTCTGCAGTTAGTAGCCATGTGAATCAGAGACCAAGCACA ...


In [2]:
from Bio import Entrez, SeqIO
from Bio.Align import substitution_matrices
import numpy as np

# =============================================
# 1. Téléchargement des séquences depuis NCBI
# =============================================
Entrez.email = "alaegmr98@gmail.com"

def get_sequence(accession):
    print(f"🔍 Téléchargement de {accession} depuis NCBI...")
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    seq = str(record.seq)
    print(f"✅ Séquence {accession} récupérée ({len(seq)} bases).")
    return seq

seq1 = get_sequence("NM_001101.5")
seq2 = get_sequence("AC035147.4")

# =============================================
# 2. Visualiser la matrice BLOSUM62
# =============================================
print("\n=== MATRICE BLOSUM62 ===")
blosum62 = substitution_matrices.load("BLOSUM62")
print(blosum62)

# =============================================
# 3. Définir la matrice de similarité personnalisée avec gap inclus
# =============================================
custom_matrix_with_gap = {
    '-': {'-': 0,  'A': -3, 'C': -3, 'G': -3, 'T': -3},
    'A': {'-': -3, 'A': 2,  'C': -1, 'G': -1, 'T': 0},
    'C': {'-': -3, 'A': -1, 'C': 2,  'G': 0,  'T': -1},
    'G': {'-': -3, 'A': -1, 'C': 0,  'G': 2,  'T': -1},
    'T': {'-': -3, 'A': 0,  'C': -1, 'G': -1, 'T': 2},
}

# =============================================
# 4. Fonctions d’alignement global (Needleman–Wunsch)
# =============================================
def alignement_global(seq1, seq2, score_table):
    n, m = len(seq1), len(seq2)
    F = np.zeros((n+1, m+1))

    # Initialisation avec gap intégré
    for i in range(1, n+1):
        F[i][0] = F[i-1][0] + score_table[seq1[i-1]]['-']
    for j in range(1, m+1):
        F[0][j] = F[0][j-1] + score_table['-'][seq2[j-1]]

    # Remplissage
    for i in range(1, n+1):
        for j in range(1, m+1):
            match = F[i-1][j-1] + score_table[seq1[i-1]][seq2[j-1]]
            delete = F[i-1][j] + score_table[seq1[i-1]]['-']
            insert = F[i][j-1] + score_table['-'][seq2[j-1]]
            F[i][j] = max(match, delete, insert)
    return F

def alignement_global_result(seq1, seq2, score_table):
    F = alignement_global(seq1, seq2, score_table)
    i, j = len(seq1), len(seq2)
    align1, align2 = "", ""

    # Backtracking
    while i > 0 and j > 0:
        score_current = F[i][j]
        if score_current == F[i-1][j-1] + score_table[seq1[i-1]][seq2[j-1]]:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif score_current == F[i-1][j] + score_table[seq1[i-1]]['-']:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1

    while i > 0:
        align1 = seq1[i-1] + align1
        align2 = '-' + align2
        i -= 1
    while j > 0:
        align1 = '-' + align1
        align2 = seq2[j-1] + align2
        j -= 1

    print("\n=== ALIGNEMENT GLOBAL ===")
    print(align1[:100] + "...")
    print(align2[:100] + "...")
    print(f"\n✅ Score global : {F[len(seq1)][len(seq2)]}")
    return F[len(seq1)][len(seq2)]

# =============================================
# 5. Tester l’alignement (réduction à 50 bases)
# =============================================
seq1_test = seq1[:50]
seq2_test = seq2[:50]

score = alignement_global_result(seq1_test, seq2_test, custom_matrix_with_gap)
print(f"\n✅ Score d’alignement (50 bases, avec gap inclus) : {score}")


🔍 Téléchargement de NM_001101.5 depuis NCBI...
✅ Séquence NM_001101.5 récupérée (1812 bases).
🔍 Téléchargement de AC035147.4 depuis NCBI...
✅ Séquence AC035147.4 récupérée (104940 bases).

=== MATRICE BLOSUM62 ===
#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.

In [4]:
from Bio import pairwise2

# =============================================
# 6. Comparaison avec Biopython : alignement global
# =============================================
match = 1       # Identité
mismatch = -1   # Substitution
gap_open = -2   # Pénalité d'ouverture de gap
gap_extend = -1 # Pénalité d'extension de gap

alignments_global = pairwise2.align.globalms(seq1_test, seq2_test, match, mismatch, gap_open, gap_extend)
best_global = alignments_global[0]

print("\n=== ALIGNEMENT GLOBAL avec Biopython ===")
print(best_global.seqA[:40] + "...")
print(best_global.seqB[:40] + "...")
print(f"Score : {best_global.score}")

# =============================================
# 7. Fonctions d'alignement local (Smith–Waterman)
# =============================================
def alignement_local(seq1, seq2, score_table):
    n, m = len(seq1), len(seq2)
    F = np.zeros((n+1, m+1))

    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = F[i-1][j-1] + score_table[seq1[i-1]][seq2[j-1]]
            delete = F[i-1][j] + score_table[seq1[i-1]]['-']
            insert = F[i][j-1] + score_table['-'][seq2[j-1]]
            F[i][j] = max(0, match_score, delete, insert)  # max avec 0 pour local
    return F

def alignement_local_result(seq1, seq2, score_table):
    F = alignement_local(seq1, seq2, score_table)
    i, j = np.unravel_index(F.argmax(), F.shape)
    align1, align2 = "", ""

    # Backtracking jusqu'à score = 0
    while i > 0 and j > 0 and F[i][j] > 0:
        score_current = F[i][j]
        if score_current == F[i-1][j-1] + score_table[seq1[i-1]][seq2[j-1]]:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif score_current == F[i-1][j] + score_table[seq1[i-1]]['-']:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1

    print("\n=== ALIGNEMENT LOCAL ===")
    print(align1[:100] + "...")
    print(align2[:100] + "...")
    print(f"Score du meilleur alignement local : {F.max()}")
    return F.max()

# =============================================
# 8. Tester l'alignement local sur les 50 premières bases
# =============================================
score_local = alignement_local_result(seq1_test, seq2_test, custom_matrix_with_gap)
print(f"\n✅ Score d’alignement local (50 bases) : {score_local}")

# =============================================
# 9. Comparaison avec Biopython : alignement local
# =============================================
alignments_local = pairwise2.align.localms(seq1_test, seq2_test, match, mismatch, gap_open, gap_extend)
best_local = alignments_local[0]

print("\n=== ALIGNEMENT LOCAL avec Biopython ===")
print(best_local.seqA[:100] + "...")
print(best_local.seqB[:100] + "...")
print(f"Score : {best_local.score}")



=== ALIGNEMENT GLOBAL avec Biopython ===
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCG...
A-AG-CTTTCCCTATTTAAAAATGTATTGTTAAAGATTTG...
Score : -18.0

=== ALIGNEMENT LOCAL ===
AGCCTCGCCTTT...
AGCTTTCCCTAT...
Score du meilleur alignement local : 14.0

✅ Score d’alignement local (50 bases) : 14.0

=== ALIGNEMENT LOCAL avec Biopython ===
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGA--------------------------------------...
--------------------------------------AAGCTTTCCCTATTTAAAAATGTATTGTTAAAGATTTGCTACGTGCCAGA...
Score : 5.0
